In [1]:
from __future__ import division
from os.path import basename, splitext
import glob
import tifffile as tiff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
in_file = "data/test1.tif"

In [7]:
with tiff.TiffFile(in_file) as tif:
        images = tif.asarray()

ch1 = 0
ch2 = 1

img_shape = images[0].shape

def imgToDataframe():
    """Function to transform all channels of open tif connection to columns in a DataFrame."""
    d = {}
    for i, data in enumerate(images):
        d["chan{}".format(i + 1)] = images[i].flatten()
    return pd.DataFrame(d)

df = imgToDataframe()

df_filter = df.copy()

df_filter[(df_filter.iloc[:, ch1] == 0) & (df_filter.iloc[:, ch2] == 0)] = np.nan

df_fix = df_filter - df_filter.min()

df_fix[(df_fix.iloc[:, ch1] < (0 * df_fix.iloc[:, ch1].max())) &
       (df_fix.iloc[:, ch2] < (0 * df_fix.iloc[:, ch2].max()))] = np.nan

# if (df_fix.iloc[:, ch1].max() >= 65535) | (df_fix.iloc[:, ch2].max() >= 65535):
#     df_fix[(df_fix.iloc[:, ch1] >= 65535) | (df_fix.iloc[:, ch2] >= 65535) ] = np.nan
# elif (df_fix.iloc[:, ch1].max() >= 4095) | (df_fix.iloc[:, ch2].max() >= 4095):
#     df_fix[(df_fix.iloc[:, ch1] >= 4095) | (df_fix.iloc[:, ch2] >= 4095) ] = np.nan
# elif (df_fix.iloc[:, ch1].max() >= 255) | (df_fix.iloc[:, ch2].max() >= 255):
#     df_fix[(df_fix.iloc[:, ch1] >= 255) | (df_fix.iloc[:, ch2] >= 255) ] = np.nan

df_mean = df_fix / np.mean(df_fix)

df_X = df_mean.iloc[:, ch1].dropna().values.reshape(-1, 1)
df_y = df_mean.iloc[:, ch2].dropna().values.reshape(-1, 1)

lm = LinearRegression()
lm.fit(X=df_X, y=df_y)
predictions = lm.predict(df_X)
rsquared = r2_score(df_y, predictions)
coef = lm.coef_

residuals = df_y - predictions

img_name = splitext(basename(in_file))[0]

In [15]:
residuals_reshaped = residuals.reshape(img_shape)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [24]:
np.argwhere(np.isnan(df_mean.iloc[:, 0]))

array([[    0],
       [    1],
       [    2],
       ..., 
       [93402],
       [93403],
       [93404]], dtype=int64)